In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/myModel

/content/drive/My Drive/myModel


In [8]:
import numpy as np
import pandas as pd
import matplotlib as plt
from sklearn.model_selection import train_test_split
import sklearn
import resnet
import os
from keras.callbacks import EarlyStopping
import keras

## **데이터 불러오기**

In [4]:
def read_dataset(root_dir, dataset_name):
    datasets_dict = {}

    df = np.loadtxt('./ALL_data.csv', delimiter=',')

    x_train, x_test, y_train, y_test = train_test_split(df[:,1:], df[:,0], test_size=0.3)
    print("x_train:", x_train.shape)
    print("x_test: ", x_test.shape)
    print("y_train", y_train.shape)
    print("y_test: ", y_test.shape)

    # znorm
    std_ = x_train.std(axis=1, keepdims=True)
    std_[std_ == 0] = 1.0
    x_train = (x_train - x_train.mean(axis=1, keepdims=True)) / std_
    std_ = x_test.std(axis=1, keepdims=True)
    std_[std_ == 0] = 1.0
    x_test = (x_test - x_test.mean(axis=1, keepdims=True)) / std_
    datasets_dict[dataset_name] = (x_train.copy(), y_train.copy(), x_test.copy(),
                                    y_test.copy())
    return datasets_dict

datasets_dict= read_dataset('', 'handcream')

x_train: (1770, 2000)
x_test:  (759, 2000)
y_train (1770,)
y_test:  (759,)


## **폴더 만들기**
#### 모델 실행 후 만들어질 파일들이 저장될거임

In [5]:
def create_directory(directory_path):
    if os.path.exists(directory_path):
        return None
    else:
        try:
            os.makedirs(directory_path)
        except:
            return None
        return directory_path

## **원핫인코딩**

In [6]:
dataset_name = 'handcream'

x_train = datasets_dict[dataset_name][0]
y_train = datasets_dict[dataset_name][1]
x_test = datasets_dict[dataset_name][2]
y_test = datasets_dict[dataset_name][3]
print("x_train:", x_train.shape)
print("x_test: ", x_test.shape)
print("y_train", y_train.shape)
print("y_test: ", y_test.shape)

nb_classes = len(np.unique(np.concatenate((y_train, y_test), axis=0)))

enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
enc.fit(np.concatenate((y_train, y_test), axis=0).reshape(-1, 1))
y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
y_test = enc.transform(y_test.reshape(-1, 1)).toarray()

y_true = np.argmax(y_test, axis=1)

if len(x_train.shape) == 2:
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
    x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

input_shape = x_train.shape[1:]

output_directory = './resnet_' + dataset_name + '/'
create_directory(output_directory)

x_train: (1770, 2000)
x_test:  (759, 2000)
y_train (1770,)
y_test:  (759,)


## **ResNet 모델**

In [9]:
n_feature_maps = 64

input_layer = keras.layers.Input(input_shape)

# BLOCK 1

conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=32, padding='same')(input_layer) #8
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)

conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=16, padding='same')(conv_x) #5
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)

conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(conv_y) #3
conv_z = keras.layers.BatchNormalization()(conv_z)

# expand channels for the sum
shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

output_block_1 = keras.layers.add([shortcut_y, conv_z])
output_block_1 = keras.layers.Activation('relu')(output_block_1)

# BLOCK 2

conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=32, padding='same')(output_block_1)
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)

conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=16, padding='same')(conv_x)
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)

conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(conv_y)
conv_z = keras.layers.BatchNormalization()(conv_z)

# expand channels for the sum
shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

output_block_2 = keras.layers.add([shortcut_y, conv_z])
print("output_block_2:", output_block_2)
output_block_2 = keras.layers.Activation('relu')(output_block_2)

# BLOCK 3

conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=32, padding='same')(output_block_2)
conv_x = keras.layers.BatchNormalization()(conv_x)
conv_x = keras.layers.Activation('relu')(conv_x)

conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=16, padding='same')(conv_x)
conv_y = keras.layers.BatchNormalization()(conv_y)
conv_y = keras.layers.Activation('relu')(conv_y)

conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(conv_y)
conv_z = keras.layers.BatchNormalization()(conv_z)

# no need to expand channels because they are equal
shortcut_y = keras.layers.BatchNormalization()(output_block_2)

output_block_3 = keras.layers.add([shortcut_y, conv_z])
output_block_3 = keras.layers.Activation('relu')(output_block_3)
# FINAL

gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)
output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

model = keras.models.Model(inputs=input_layer, outputs=output_layer)

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(),
                metrics=['accuracy'])

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50, min_lr=0.0001)

file_path = output_directory + 'best_model.hdf5'

early_stopping_callback = EarlyStopping(monitor = 'loss', mode='min', patience=10, verbose=1)

model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
                                                    save_best_only=True)

callbacks = [reduce_lr, model_checkpoint, early_stopping_callback]


output_block_2: KerasTensor(type_spec=TensorSpec(shape=(None, 2000, 128), dtype=tf.float32, name=None), name='add_1/add:0', description="created by layer 'add_1'")


In [ ]:
model.summary()

## **학습**

In [11]:
# validation_split: train dataset에서 validation dataset으로 분리할 크기 
# 0.2 => 20%
history = model.fit(x_train, y_train, batch_size=16, epochs=64, validation_split=0.3, callbacks=callbacks)

Epoch 1/64
78/78 [==============================] - 9s 120ms/step - loss: 1.7820 - accuracy: 0.3172 - val_loss: 2.9414 - val_accuracy: 0.1431


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/64
78/78 [==============================] - 8s 109ms/step - loss: 1.6682 - accuracy: 0.3745 - val_loss: 4.1671 - val_accuracy: 0.1394
Epoch 3/64
78/78 [==============================] - 8s 109ms/step - loss: 1.6367 - accuracy: 0.3939 - val_loss: 3.7492 - val_accuracy: 0.1601
Epoch 4/64
78/78 [==============================] - 9s 109ms/step - loss: 1.5773 - accuracy: 0.4068 - val_loss: 1.7167 - val_accuracy: 0.3672
Epoch 5/64
78/78 [==============================] - 9s 110ms/step - loss: 1.4956 - accuracy: 0.4149 - val_loss: 2.4678 - val_accuracy: 0.2279
Epoch 6/64
78/78 [==============================] - 9s 111ms/step - loss: 1.4636 - accuracy: 0.4431 - val_loss: 3.2264 - val_accuracy: 0.1977
Epoch 7/64
78/78 [==============================] - 9s 111ms/step - loss: 1.3832 - accuracy: 0.4851 - val_loss: 2.0043 - val_accuracy: 0.2863
Epoch 8/64
78/78 [==============================] - 9s 111ms/step - loss: 1.3346 - accuracy: 0.4867 - val_loss: 1.9221 - val_accuracy: 0.3352
Epoch 

## **모델 평가**

In [ ]:
import matplotlib.pyplot as plt
import keras
import sklearn

model_path = output_directory + 'best_model.hdf5'
model = keras.models.load_model(model_path)

print("정확도 : %.2f " %(model.evaluate(x_test, y_test)[1]))

prediction = model.predict(X)

for i in prediction:
    pre_ans = i.argmax()  # 예측 레이블
    print(i)
    print(pre_ans)

y_pred = model.predict(x_test)
print(y_pred)
y_pred = np.argmax(y_pred, axis=1)

## **시각화**

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss', 'acc', 'val_acc'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.savefig('loss.png')
plt.clf()
plt.plot(history.history['val_loss'])
plt.savefig('vloss.png')
plt.clf()
plt.plot(history.history['accuracy'])
plt.savefig('acc.png')
plt.clf()
plt.plot(history.history['val_accuracy'])
plt.savefig('vacc.png')
plt.clf()
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['loss', 'accuracy', 'val_accuracy'], loc='upper left')
# plt.show()